In [ ]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

In [ ]:
mongo_client = MongoClient(os.environ["MONGO_CONNECTION_STRING"])

In [ ]:
user_db = mongo_client["users"]["user-data"]

In [ ]:
cursor = user_db.find({})

In [ ]:
old_moments = user["moments"]

In [ ]:
for user in cursor:
    _id = user["company_id"]
    new_moments = {
        "General News":["A", "bf", "XC"],
        "Industry News":["A", "bfa", "XC"],
        "Current Events":["A", "basd", "XC"],
        "social_media":["A", "b", "XCsad"],
    }

    new_values = {"$set": {"moments": new_moments}}
    print(user_db.update_one({"company_id":_id}, new_values))


In [ ]:
from chroma_interface.reader import Reader
from chroma_interface.writer import Writer
import os

keys = list(open(".env", "r"))

for item in keys:
    variable, value = item.split("=")[0], "=".join(item.split("=")[1:])
    os.environ[variable] = value.replace("\n", "")

In [ ]:
openai_key = os.environ["OPENAI_API_KEY"]
chroma_host = os.environ["CHROMA_IP"]

reader = Reader(host=chroma_host, port=8000, collection="test-1", openai_api_key=openai_key)
writer = Writer(host=chroma_host, port=8000, openai_api_key=openai_key)

In [ ]:
collections = reader.list_collections()

In [1]:
from utils.simple_utils import *
from utils.google_utils import *
from utils.langchain_utils import *
from utils.best_hashtags import *

from mongo.interface import *

from chroma_interface import *
from chroma_interface.reader import *
from chroma_interface.writer import *

import os
import time
# Constants
NEWS_TOPICS = read_lines_from_file("./config/topic-type.txt")

COUNTRY_NAMES = read_lines_from_file("./config/countries-names.txt")
COUNTRY_CODES = defaultdict(None)
for name, code in zip(COUNTRY_NAMES, read_lines_from_file("./config/countries-code.txt")):
    COUNTRY_CODES[name] = code

COUNTRY_CODES_TO_NAMES = defaultdict(None)
for name, code in zip(COUNTRY_NAMES, read_lines_from_file("./config/countries-code.txt")):
    COUNTRY_CODES_TO_NAMES[code] = name

QUERY_KEYWORDS_DICT = json.load(open("./config/query-topics.json", 'r'))
QUERY_TOPICS = list(QUERY_KEYWORDS_DICT.keys())

QUERY_KEYWORDS = []
for item in QUERY_KEYWORDS_DICT.values():
    QUERY_KEYWORDS.extend(item)


# environment setup
with open(".env", "r") as key_file:
    keys = list(key_file)

for item in keys:
    variable, value = item.split("=")[0], "=".join(item.split("=")[1:])
    os.environ[variable] = value.replace("\n", "")

topic=NEWS_TOPICS[0]
country_name = "India"
country_code=COUNTRY_CODES[country_name]
lang='en'
collection_name = "general_news"

chroma_reader, chroma_writer = get_reader_writer(
    host=os.environ["CHROMA_IP"],
    port=int(os.environ["CHROMA_PORT"]),
    openai_api_key=os.environ["OPENAI_API_KEY"],
    reader_collection_name=collection_name
)

user_mongo_client = MongoInterface(
    url=os.environ["MONGO_CONNECTION_STRING"],
    database="users",
    collection="user-data"
)

from scripts import *


In [2]:
_id = 3894336111
user_data = user_mongo_client.get_user(_id)

In [3]:
content_category = user_data['content_category']
company_description = user_data['company_description']
query = content_category + "|" + company_description

In [4]:
content_category

' Cosmetics, Skin Care, Make-up and Beauty.'

In [5]:
filter_news(content_category, chroma_reader, country_code)

[{'title': 'Mini Vacuum Cleaners: To Clean Furniture and Car Interiors In a Simple Way - IndiaTimes',
  'description': 'Most Searched Products News: Here is a compiled list of some of the best quality and affordable mini vacuum cleaners available online at a discount which you cannot afford to miss out on. (December, 2023)',
  'url': 'https://news.google.com/rss/articles/CBMisQFodHRwczovL20udGltZXNvZmluZGlhLmNvbS9tb3N0LXNlYXJjaGVkLXByb2R1Y3RzL2VsZWN0cm9uaWNzL3ZhY3V1bS1jbGVhbmVycy9taW5pLXZhY3V1bS1jbGVhbmVycy10by1jbGVhbi1mdXJuaXR1cmUtYW5kLWNhci1pbnRlcmlvcnMtaW4tYS1zaW1wbGUtd2F5L2FydGljbGVzaG93LzEwNTczMzk3MC5jbXPSAbUBaHR0cHM6Ly9tLnRpbWVzb2ZpbmRpYS5jb20vbW9zdC1zZWFyY2hlZC1wcm9kdWN0cy9lbGVjdHJvbmljcy92YWN1dW0tY2xlYW5lcnMvbWluaS12YWN1dW0tY2xlYW5lcnMtdG8tY2xlYW4tZnVybml0dXJlLWFuZC1jYXItaW50ZXJpb3JzLWluLWEtc2ltcGxlLXdheS9hbXBfYXJ0aWNsZXNob3cvMTA1NzMzOTcwLmNtcw?oc=5',
  'card_text': 'Listed below are some of the best mini vacuum cleaners available online: Ambrane Portable Cordless Vacuum Cleane

In [15]:
client = HttpClient(
    host=os.environ["CHROMA_IP"],
    port=int(os.environ["CHROMA_PORT"])
)
collection = client.get_collection(collection_name, embedding_function=OpenAIEmbeddingFunction(api_key=os.environ["OPENAI_API_KEY"]))

In [22]:
outs = collection.query(query_texts=query, n_results=20, where={"country_code":country_code})

In [8]:
chroma_reader.search(query, )

[Document(page_content='Lunar Mysteries: From ancient calendars to 21st Century exploration - The Financial Express', metadata={'card_text': 'Earth’s sole satellite, measuring just a quarter of its size, traverses the space around our home planet in a state...', 'country_code': 'IN', 'description': 'The initial era of lunar exploration unfolded amid the geopolitical competition of the Cold War between the United States and the Soviet Union.', 'keywords': "Lunar Mysteries,Moon Exploration,Earth's sole satellite,Solar System", 'source': 'The Financial Express', 'source_href': 'https://www.financialexpress.com', 'text': 'Earth’s sole satellite, measuring just a quarter of its size, traverses the space around our home planet in a state of cold and lifeless isolation. Its orbit expands by an inch and a half each year.\n\nThe Moon has been Earth’s companion for approximately 4.6 billion years, since the birth of the Solar System. In ancient times, humans were fascinated by the Moon’s ever-ch

In [2]:
chroma_reader.set_collection("current_events")

In [3]:
query = f"""List 10 important and interesting events related to any of these topics: {','.join(QUERY_TOPICS)}. 
Zomato is a popular online platform that provides a comprehensive range of services related to dining and food. Originally founded in India, Zomato has expanded globally and is now a prominent player in the food technology industry. The platform allows users to explore and discover restaurants, cafes, and eateries in their vicinity, offering detailed information such as menus, reviews, ratings, and photos.

Zomato's user-friendly interface enables individuals to browse through a diverse range of cuisines, make reservations, and order food for delivery or takeaway. The platform has become a go-to resource for food enthusiasts seeking recommendations, as it aggregates user-generated reviews to help people make informed decisions about where to dine.

In addition to its consumer-facing services, Zomato has also ventured into food delivery, connecting users with a wide network of restaurants and delivery partners. The platform's commitment to enhancing the overall dining experience has made it a popular choice for both food lovers and businesses in the food industry."""


In [4]:
topic = "film industry"
target = "text"
country_code = "IN"

In [5]:
chroma_reader.search(query, filter=dict(country_code=country_code))

[Document(page_content="capitalizing on the online gaming realm for product launches, events, and innovative monetization avenues. Notable entries include United Colors of Benetton's virtual collection in Animal Crossing and Toyota's engagement with its Asian audience through the e-Motorsport event TOYOTA GAZOO Racing GT Cup on Gran Turismo, live-streamed on Twitch, Facebook, YouTube, and Twitter, allowing gamers to race using Toyota sports cars. Prominent startups in the gaming sector, including MPL, Dream Sports, and Games 24 Seven, have achieved unicorn status. Nazara stands out as the first listed gaming company. Noteworthy international gaming firms like Electronic Arts (FIFA), Rockstar Games (Call of Duty), Ubisoft, and Zynga also have a significant presence in India. The rise of gaming culture is undeniably shaping a new era of entertainment and the government's role in fostering innovation plays a pivotal part in this transformative journey. If stakeholders and the government c

In [3]:
client = HttpClient(host=os.environ["CHROMA_IP"], port=os.environ["CHROMA_PORT"])
embedding_function=OpenAIEmbeddingFunction(os.environ["OPENAI_API_KEY"])

In [5]:
collection = client.get_collection("current_events", embedding_function=embedding_function)

In [17]:
collection.get('69c1ee9c-3b34-5468-9104-abcf6546663a')

{'ids': [], 'embeddings': None, 'metadatas': [], 'documents': []}

In [11]:
doc = "vednat is a top tier dawg. vednat is a top tier dawg vednat is a top tier dawg vednat is a top tier dawg vednat is a top tier dawg vednat is a top tier dawg vednat is a top tier dawg vednat is a top tier dawg vednat is a top tier dawg vednat is a top tier dawg vednat is a top tier dawg vednat is a top tier dawg vednat is a top tier dawg vednat is a top tier dawg vednat is a top tier dawg vednat is a top tier dawg vednat is a top tier dawg vednat is a top tier dawg vednat is a top tier dawg vednat is a top tier dawg vednat is a top tier dawg vednat is a top tier dawg vednat is a top tier dawg vednat is a top tier dawg vednat is a top tier dawg vednat is a top tier dawg vednat is a top tier dawg "

In [12]:
collection.add(ids=['08335a8d-8860-11ee-b3d3-d807c903b03b'], documents=[doc], metadatas=[{"mr":"dog"}])

In [32]:
import pickle as pkl
import numpy as np
docs_and_meta = pkl.load(open("test.pkl", "rb"))

# get links for all the topics
documents = []
metadata = []
ids = []

for temp_data in docs_and_meta:
    data = np.array(temp_data, dtype=object).T

    ids.extend(list(map(lambda x: get_id(str(x[0])+str(x[1])), data)))
    documents.extend(list(data.T[0]))
    metadata.extend(list(data.T[1]))

temp = pd.DataFrame({
    "ids": ids,
    "documents": documents,
    "metadata": metadata
})

print("Total documents:", len(temp))
temp.drop_duplicates("documents", inplace=True)
print("Total documents after removing duplicates:", len(temp))

Total documents: 4630
Total documents after removing duplicates: 3754


In [33]:
import pickle as pkl
import numpy as np
docs_and_meta = pkl.load(open("test.pkl", "rb"))

# get links for all the topics
documents = []
metadata = []
ids = []

for temp_data in docs_and_meta:
    data = np.array(temp_data, dtype=object).T

    ids.extend(list(map(lambda x: get_id(str(x[0])+str(x[1])), data)))
    documents.extend(list(data.T[0]))
    metadata.extend(list(data.T[1]))

temp = pd.DataFrame({
    "ids": ids,
    "documents": documents,
    "metadata": metadata
})

print("Total documents:", len(temp))
temp.drop_duplicates("ids", inplace=True)
print("Total documents after removing duplicates:", len(temp))

Total documents: 4630
Total documents after removing duplicates: 3772


In [14]:
collection.add(ids=[str(uuid.uuid5(uuid.NAMESPACE_DNS, doc))], documents=[doc], metadatas=[{"mr":"dog"}])


In [13]:
str(uuid.uuid5(uuid.NAMESPACE_DNS, doc))

'2f9bfe5a-4261-5404-93aa-38b07b88ad2a'

In [13]:
results = collection.query(query_texts=query)

In [14]:
docs = results['documents'][0]
ids = results['ids'][0]
meta = results['metadatas'][0]

In [24]:
get_id(str(docs[0]) + str(meta[0]))

'49992438-54fb-5a27-a855-b40e0f6f8642'

In [23]:
get_id(str(docs[1]) + str(meta[1]))

'32c2533f-c9da-5ebf-aa4e-f63369223d64'

In [26]:
import difflib

cases=[(str(meta[0]), str(meta[1]))] 

for a,b in cases:     
    print('{} => {}'.format(a,b))  
    for i,s in enumerate(difflib.ndiff(a, b)):
        if s[0]==' ': continue
        elif s[0]=='-':
            print(u'Delete "{}" from position {}'.format(s[-1],i))
        elif s[0]=='+':
            print(u'Add "{}" to position {}'.format(s[-1],i))    
    print()  

{'card_text': 'Even when attending live sporting events in person, Gen Z fans are still focused on connecting with others and are...', 'country_code': 'IN', 'description': "Sports fandom is changing. A more immersive sports fan experience involving multiple channels and modes of engagement will best meet the changing fan's needs.", 'keywords': '', 'source': 'Deloitte', 'source_href': 'https://www2.deloitte.com', 'title': 'Immersive sports fandom - Deloitte', 'top_image': 'https://assets.deloitte.com/is/image/deloitte/US176271_Banner-1:1200-x-627', 'url': 'https://news.google.com/rss/articles/CBMiZmh0dHBzOi8vd3d3Mi5kZWxvaXR0ZS5jb20vdWsvZW4vaW5zaWdodHMvaW5kdXN0cnkvbWVkaWEtYW5kLWVudGVydGFpbm1lbnQvaW1tZXJzaXZlLXNwb3J0cy1mYW5kb20uaHRtbNIBAA?oc=5'} => {'card_text': 'Even when attending live sporting events in person, Gen Z fans are still focused on connecting with others and are...', 'country_code': 'IN', 'description': "Sports fandom is changing. A more immersive sports fan experience invol

In [28]:
str(meta[0])[635:655]

'b20vdWsvZW4vaW5zaWdo'

In [29]:
str(meta[1])[635:655]


'b20vdXMvZW4vaW5zaWdo'

In [ ]:
users = user_mongo_client.get_user_list()

for user in users:
    _id = user["company_id"]
    old_moments = user["moments"]

    new_current_events = []

    for item in old_moments["Current Events"]:
        new_current_events.append({
            "event_name": item["event_name"],
            "topic": item["topic"],
            "validation": {
                "google_trends": item["validation"]["google_trends"]["keywords"]
            }
        })

In [ ]:
old_moments["Current Events"] = new_current_events

In [ ]:
user_mongo_client.update_user_moments(100, old_moments)

In [ ]:
chroma_reader, chroma_writer = get_reader_writer(
    host=os.environ["CHROMA_IP"],
    port=int(os.environ["CHROMA_PORT"]),
    openai_api_key=os.environ["OPENAI_API_KEY"],
    reader_collection_name=collection_name
)

In [ ]:
chroma_reader.list_collections()

In [ ]:
collection_name="current_events"
topic="entertainment"

In [ ]:
create_new_collection(chroma_reader, chroma_writer, collection_name)

In [ ]:
news_df = get_news_by_search(f"upcoming or ongoing major events about {topic}", limit=50, country=country_code)
news_df.to_csv("./test/news.csv", index=False)

In [ ]:
news_df = pd.read_csv("./test/news.csv")

In [ ]:
article = parse_news_url(news_df["url"][0])

In [ ]:
target = "text"

news_df = news_df[(news_df["text"].isna() == False)]
news_df["country_code"] = [country_code] * len(news_df)


In [ ]:
def split_df(df:pd.DataFrame, target:str) -> tuple[list[str], list[dict]]:
    texts = df[target].values

    meta_columns = df.columns.to_list()
    meta_columns.remove(target)

    meta_df = df[meta_columns]

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separators=[" ", ".", ",", "\n"])

    documents, metadata = [], []

    for i, item in enumerate(texts):
        splits = text_splitter.split_text(item)
        documents.extend(splits)
        metadata.extend([meta_df.iloc[i].to_dict()] * len(splits))

    return documents, metadata


In [ ]:
documents, metadata = split_df(news_df, target)

In [ ]:
chroma_writer.update(collection_name, documents, metadata)

In [ ]:
target = "text"
df=news_df.copy()
texts = df[target].values
meta_columns = df.columns.to_list()
meta_columns.remove(target)
metadata = df[meta_columns]

In [ ]:
chroma_reader.set_collection(collection_name)

In [ ]:
current_events_moments = generate_current_events(chroma_reader, QUERY_TOPICS, country_code)

In [ ]:
query = f"List 10 important and interesting events related to any of these topics: {','.join(QUERY_TOPICS)}."

relevant_docs = chroma_reader.search(query, 30, {"country_code": country_code})

In [ ]:
metadata.iloc[0].to_dict()

In [ ]:
news_df

In [ ]:
documents, metadata = load_df(news_df, page_content_column="text")


In [ ]:
get_current_events()

In [ ]:
collection_name = "current_events"

chroma_reader.set_collection(collection_name)

current_events_moments = generate_current_events(chroma_reader,QUERY_TOPICS)

In [ ]:
current_events_moments

In [ ]:
googleSearch(f"upcoming or ongoing major events about sports tournaments", num_results=20, country=country_code)

In [ ]:
get_news_by_search(f"upcoming or ongoing major events about sports tournaments", country=country_code, limit=20)

In [ ]:
chroma_reader.search(f"List 10 important and interesting events related to any of these topics: {','.join(QUERY_TOPICS)}.", n=30, filter={"country_code":"IN"})

In [ ]:
relevant_docs

In [ ]:
current_events_moments

In [ ]:
get_metadata(article)

In [ ]:
temp_links = [('https://www.visitphilly.com/articles/philadelphia/top-events-and-festivals-in-philadelphia/', 'IN'), ('https://en.wikipedia.org/wiki/News', 'IN'), ('https://www.nih.gov/about-nih/what-we-do/nih-almanac/national-institute-drug-abuse-nida', 'IN'), ('https://www.cnn.com/', 'IN'), ('https://blog.twitter.com/en_us/topics/insights/2022/how-many-people-come-twitter-for-news', 'IN')]
articles_data = [parse_for_current_events(item) for item in temp_links]

In [ ]:
target_key="source"

In [ ]:
docs, metadata = build_splited_docs(articles_data, target_key="source")

In [ ]:
metadata

In [ ]:
page = articles_data[0]

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separators=[" ", ".", ",", "\n"])


In [ ]:
splits = text_splitter.split_text(page[target_key])


In [ ]:
page.pop(target_key, None)

In [ ]:
page

In [ ]:
get_current_events()

In [ ]:
create_new_collection(chroma_reader, chroma_writer, collection_name)

In [ ]:
def build_splited_docs(sitetexts):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separators=[" ", ".", ",", "\n"])

    docs, metadatas = [], []
    for page in sitetexts:
        splits = text_splitter.split_text(page['text'])
        docs.extend(splits)
        metadatas.extend([{"source": page['source'], "country_code": page["country_code"]}] * len(splits))

    return docs, metadatas

In [ ]:
import pickle as pkl

In [ ]:
(docs, metadata) = pkl.load(open("test.pkl", "rb"))

In [ ]:
chunked_docs = divide_chunks(docs, 1000)
chunked_metadata = divide_chunks(metadata, 1000)

In [ ]:
for docs, metadata in zip(chunked_docs, chunked_metadata):
    chroma_writer.update("test_industry", docs, metadata)

In [ ]:
chroma_writer.delete_collection("general_news")

In [ ]:
chroma_reader.list_collections()

In [ ]:
chroma_reader.set_collection("test_industry")

In [ ]:
outputs = chroma_reader.search("Indian Food", n=40)

In [ ]:
news_df = get_news_by_topic("SPORTS", country=country_code, lang=lang)

In [ ]:
news_df = news_df[(news_df["top_image"] != "") & (news_df["keywords"] != "")]


In [ ]:
article = parse_news_url(news_df["url"][0])

In [ ]:
news_df.iloc[0]

In [ ]:
news_df["keywords"][0]

In [ ]:
article.meta_keywords

In [ ]:
def load_df(df, page_content_column="title"):
    data = DataFrameLoader(df, page_content_column=page_content_column).load()

    documents = [item.page_content for item in data]
    metadata = [item.metadata for item in data]

    return documents, metadata

In [ ]:
documents, metadata = load_df(news_df)

In [ ]:
chroma_writer.update("general_news", documents, metadata)

In [ ]:
article = parse_news_url("https://news.google.com/rss/articles/CBMic2h0dHBzOi8vd3d3Lndoby5pbnQvbmV3cy9pdGVtLzEwLTExLTIwMjMtY2hpbGRyZW4td2l0aC1jYW5jZXItZXZhY3VhdGVkLWZyb20tZ2F6YS1mb3ItdHJlYXRtZW50LXRvLWVneXB0LWFuZC1qb3JkYW7SAQA?oc=5")

In [ ]:
outputs = chroma_reader.search("Indian Food")[0]

In [ ]:
chroma_reader.search("Indian Food")

In [ ]:
chroma_reader.set_collection("general_news")

In [ ]:
filtered = filter_news("Diwali", chroma_reader)

In [ ]:
general = filtered[:5]
industry = filtered[5:11]

In [ ]:
user_mongo_client.update_user_moments(
    100,
    {
        "general_news": general,
        "industry": industry,
        "current_events": [
            {
                "event_name": "ICC Cricket world cup 2023",
                "topic": "Cricket world cup hosted in India",
                "validation": {
                    "google_trends": {
                        "keywords": ["cricket", "icc world cup", "india cricket"]
                    }
                }
            },
            {
                "event_name": "Diwali",
                "topic": "One of the biggest festivals in India",
                "validation": {
                    "google_trends": {
                        "keywords": ["diwali", "indian festivals", "lights"]
                    }
                }
            }
        ],
        "social_media": [
            {
                "title": "Run the dog trends",
                "hashtag": ["#crazydog", "#mrdoge", "#woof"],
                'validation': {
                    "hashtags": [
                        {"#crazydog": 100210,
                         "#mrdoge": 100232, 
                         "#woof": 100311}
                    ]
                }
            },
            {
                "title": "Night is day? what?",
                "hashtag": ["#idontunderstand", "#oogaboogabrain", "#what"],
                'validation': {
                    "hashtags": [
                        {"#idontunderstand": 100210,
                         "#oogaboogabrain": 100232, 
                         "#what": 100311}
                    ]
                }
            }
        ]
    }
)

In [2]:
import pandas as pd

In [3]:
df_1 = pd.read_clipboard()

In [5]:
df_2 = pd.read_clipboard()

In [7]:
df_1

,product_title,product_vendor,product_type,adjustment,cancelled,financial_status,fulfillment_status,order_id,order_name,sale_kind,...,gift_card_discounts,gift_card_gross_sales,gift_cards_issued,pending_sales,ordered_item_quantity,average_units_ordered,returned_item_quantity,percent_of_sales_with_staff_help,InsertDate,BrandName


In [6]:
df_2

,ad_id,xyz_campaign_id,fb_campaign_id,age,gender,interest,Impressions,Clicks,Spent,Total_Conversion,Approved_Conversion
0,708746,916,103916,30-34,M,15,7350,1,1.430000,2,1
1,708749,916,103917,30-34,M,16,17861,2,1.820000,2,0
2,708771,916,103920,30-34,M,20,693,0,0.000000,1,0
3,708815,916,103928,30-34,M,28,4259,1,1.250000,1,0
4,708818,916,103928,30-34,M,28,4133,1,1.290000,1,1
...,...,...,...,...,...,...,...,...,...,...,...
1138,1314410,1178,179977,45-49,F,109,1129773,252,358.189997,13,2
1139,1314411,1178,179978,45-49,F,110,637549,120,173.880003,3,0
1140,1314412,1178,179979,45-49,F,111,151531,28,40.289999,2,0
1141,1314414,1178,179981,45-49,F,113,790253,135,198.710001,8,2
